# ANÀLISIS DEL SALARIO MINIMO INTERPROFESIONAL (SMI)

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import webbrowser
from selenium import webdriver
import requests 

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
from time import sleep

In [ ]:

all_data=[]
anomaxrango=2022
anominrango=1978
preciominrango=1000
preciomaxrango=50000
rango=400
anomax=0
#Recorremos los años empezando la fecha del mayor al menor de forma inversa del mayor al menor 
for anomin in range(anomaxrango,anominrango,-1):
# definimos el el maximo apartir del año en curso minimo
    anomax = anomin + 1 
# Recorremos los precios de 400 en 400 del precio mínimo al máximo
    for preciomin in range(preciominrango,preciomaxrango, rango):
# Definimos el precio máximo apartir del minimo y el rango
        preciomax = preciomin + rango
        print(f"año{anomax}")
        print(preciomax)
# Este bucle es para recorrer las distintas página
        for i in range(1, 21):
                #print(i)
# En esta URL le metemos los distintos parametros de los bucles anteriores ya que enseña de 400 en 400
                url = f'https://www.autoscout24.es/lst?atype=C&cy=E&damaged_listing=exclude&desc=0&fregfrom={anomin}&fregto={anomax}&page={i}&powertype=kw&pricefrom={preciomin}&priceto={preciomax}&search_id=sgbo79pcq7&sort=standard&source=listpage_pagination&ustate=N%2CU'
                response = requests.get(url)
                soup = BeautifulSoup(response.text, "html.parser")
# Esta paso recoremos la clase para extraer la marca del vehículo 
                marca_elements = soup.find_all(class_="ListItem_title__znV2I ListItem_title_new_design__lYiAv Link_link__pjU1l")
                marcas = [element.find('h2').text for element in marca_elements]
# Si no tenmos marca del vehículo sideja de extraer porque no hay, pues vamos al bucle anterio
                if not marcas: 
                    #print("No marca found for this page, moving to the next iteration.")
                    break;
# extraemos precio 
                precio_elements = soup.find_all('p', attrs={"data-testid": "regular-price"})
                precios = [element.text.replace(",-", "").strip() for element in precio_elements]

                # extraemos ciudad 
                ciudad_elements = soup.find_all('span', class_="SellerInfo_address__txoNV")
                ciudades = [element.text.replace("Contáctanos en: •", "").strip() for element in ciudad_elements]

                # extraemos km, Combustible, Cambio, and anos information
                elementos = soup.find_all('span', class_="VehicleDetailTable_item__koEV4")
                km = [detail.text for detail in elementos if 'km' in detail.text]
                Combustible = [detail.text for detail in elementos if
                               'Gasolina' in detail.text or 'Diésel' in detail.text or 'Híbrido' in detail.text
                               or 'Eléctrico' in detail.text or 'Híbrido Enchufable' in detail.text
                               or 'Etanol' in detail.text or 'Gas licuado (GLP)' in detail.text
                               or 'Gas natural (CNG)' in detail.text or 'Electro/Gasolina' in detail.text
                               or 'Electro/Diésel' in detail.text or 'Otros' in detail.text]
                #Cambio = [detail.text for detail in elementos if 'Manual' in detail.text or
                         # 'Automático' in detail.text or 'Semiautomático' in detail.text]
                anos = [detail.text for detail in elementos if '/' in detail.text and len(detail.text) == 7]

                # Unimos los distintos elementos
                page_data = list(zip(marcas,Combustible,anos, km, ciudades,precios))
                print(page_data)
                all_data.extend(page_data)

with open('myfile.csv', 'w', newline='') as file:
        mywriter = csv.writer(file, delimiter=',')
        mywriter.writerows(all_data)
                      
# Imprimimos los datos
for item in all_data:
    print(item)

In [ ]:
len(all_data)

In [ ]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup

def procesar_url(url):
    response = HTMLSession().get(url)
    content = response.html.html
    soup = BeautifulSoup(content, "html.parser")
    
    car_listings = []
    
    # nos metenos dentro del contenedor donde esta el anuncio
    listing_containers = soup.find_all("div", class_="contenido-anuncio")
    for container in listing_containers:
        car_details = []
        
        #Extraemos la marca
        marca_element = container.find("h2", itemprop="name")
        if marca_element:
            car_brand = marca_element.text.strip()
            car_details.append(car_brand)
        
        # Extraemos los datos de la ul
        ul_element = container.find("ul")
        li_elements = ul_element.find_all("li")
        
        # Extraemos lo datos de li
        for li in li_elements:
            detail = li.get_text().strip()
            car_details.append(detail)
        
        # extraemos los datos de precio y no de precio financiado <p class="precio ">
        precio_financiado = container.find("p", class_="precio")
        if precio_financiado:
            precio_value = precio_financiado.find("span", recursive=False, class_=False).get_text().strip()
            car_details.append(precio_value)
        
        car_listings.append(car_details)
    
    # eliminamos el elemento 5 que es de la cilindrada
    for sublist in car_listings:
        if len(sublist) >= 5:
            sublist.pop(4)
    
    return car_listings
todo=[]
i=0
for i in range(1,5889,1):
    #print(i)
    url = f'https://www.autocasion.com/coches-ocasion?page={i}'
    results = procesar_url(url)
    for result in results:
        #print(result)
        todo.append(result)
len(todo)

In [2]:
todo

[['JEEP Compass 1.6 Mjt Longitude 4x2',
  'Diésel',
  '2019',
  '96.700 km',
  'Lleida',
  '21.900 €'],
 ['BMW X3 xDrive 20dA', 'Diésel', '2020', '46.776 km', 'Madrid', '42.995 €'],
 ['KIA Stonic 1.0 T-GDi MHEV Drive 100',
  'Gasolina',
  '2022',
  '16.600 km',
  'Huesca',
  '19.550 €'],
 ['MINI Countryman Todoterreno  Automático de 5 Puertas',
  'Diésel',
  '2016',
  '30.000 km',
  'Córdoba',
  '27.900 €'],
 ['SEAT Ibiza 1.0 TSI S&S Style 110',
  'Gasolina',
  '2021',
  '54.800 km',
  'Pontevedra',
  '16.400 €'],
 ['MERCEDES-BENZ Clase GLC 4Matic',
  'Diésel',
  '2019',
  '123.150 km',
  '35.900 €'],
 ['VOLKSWAGEN Golf 2.0 TSI GTI Clubsport DSG 221kW',
  'Gasolina',
  '2021',
  '20.000 km',
  'Barcelona',
  '43.990 €'],
 ['PEUGEOT 2008 1.5BlueHDi S&S Allure Pack 110',
  'Diésel',
  '2022',
  '19.000 km',
  'Málaga',
  '25.490 €'],
 ['MERCEDES-BENZ Clase C 200 9G-Tronic',
  'Gasolina',
  '2019',
  '74.865 km',
  'Málaga',
  '30.900 €'],
 ['RENAULT Scénic Grand 1.5dCi Intens 81kW',
  'D

In [7]:
nombre_archivo = 'coches3.csv'

# Abrir el archivo CSV en modo escritura
with open(nombre_archivo, 'w', newline='') as archivo_csv:
    # Crear un objeto de escritura CSV
    escritor_csv = csv.writer(archivo_csv)
    
    # Escribir los datos en el archivo CSV
    escritor_csv.writerows(todo)

print("Archivo CSV creado:", nombre_archivo)

Archivo CSV creado: coches3.csv


In [49]:
columnas = ['marcas', 'Combustible', 'anos', 'km', 'ciudades', 'precios']

# Crear el DataFrame
df = pd.DataFrame(todo, columns=columnas)

# Imprimir el DataFrame
df

,marcas,Combustible,anos,km,ciudades,precios
0,JEEP Compass 1.6 Mjt Longitude 4x2,Diésel,2019,96.700 km,Lleida,21.900 €
1,BMW X3 xDrive 20dA,Diésel,2020,46.776 km,Madrid,42.995 €
2,KIA Stonic 1.0 T-GDi MHEV Drive 100,Gasolina,2022,16.600 km,Huesca,19.550 €
3,MINI Countryman Todoterreno Automático de 5 P...,Diésel,2016,30.000 km,Córdoba,27.900 €
4,SEAT Ibiza 1.0 TSI S&S Style 110,Gasolina,2021,54.800 km,Pontevedra,16.400 €
...,...,...,...,...,...,...
9995,JEEP Renegade 1.3 G 110KW NIGHT EAGLE FWD DDCT 5P,Gasolina,2022,11.916 km,Barcelona,31.850 €
9996,AUDI A3 2.0 30 TDI S TRONIC SPORTBACK 5P,Diésel,2022,6.554 km,Barcelona,27.800 €
9997,FORD Puma 1997 1.0 EcoBoost 125cv ST-Line MHEV...,Gasolina,2021,24.095 km,Valencia,23.900 €
9998,VOLKSWAGEN Taigo 1.0 TSI R-Line 81kW,Gasolina,2023,8.000 km,Alicante,25.500 €


In [50]:
df['año'] = df['anos']

In [51]:
df = df.drop(columns=['anos'])

In [52]:
df['kms'] = df['km'].str.split().str[0]

In [53]:
df = df.drop(columns=['km'])

In [54]:
df['precio'] = df['precios'].str.replace(' €', '')

In [55]:
df = df.drop(columns=['precios'])

In [56]:
df['ciudad'] = df['ciudades']

In [57]:
df = df.drop(columns=['ciudades'])

In [58]:
df['marca'] = df['marcas'].str.split().str[0]

In [59]:
df['modelo'] = df['marcas'].str.split(n=1).str[1]

In [60]:
df = df.drop(columns=['marcas'])

In [64]:
df

,Combustible,año,kms,precio,ciudad,marca,modelo
0,Diésel,2019,96.700,21.900,Lleida,JEEP,Compass 1.6 Mjt Longitude 4x2
1,Diésel,2020,46.776,42.995,Madrid,BMW,X3 xDrive 20dA
2,Gasolina,2022,16.600,19.550,Huesca,KIA,Stonic 1.0 T-GDi MHEV Drive 100
3,Diésel,2016,30.000,27.900,Córdoba,MINI,Countryman Todoterreno Automático de 5 Puertas
4,Gasolina,2021,54.800,16.400,Pontevedra,SEAT,Ibiza 1.0 TSI S&S Style 110
...,...,...,...,...,...,...,...
9995,Gasolina,2022,11.916,31.850,Barcelona,JEEP,Renegade 1.3 G 110KW NIGHT EAGLE FWD DDCT 5P
9996,Diésel,2022,6.554,27.800,Barcelona,AUDI,A3 2.0 30 TDI S TRONIC SPORTBACK 5P
9997,Gasolina,2021,24.095,23.900,Valencia,FORD,Puma 1997 1.0 EcoBoost 125cv ST-Line MHEV Auto
9998,Gasolina,2023,8.000,25.500,Alicante,VOLKSWAGEN,Taigo 1.0 TSI R-Line 81kW


In [71]:
df = df.copy()

# Eliminar las columnas "ciudad" y "modelo" de la copia
columnas_a_eliminar = ["ciudad", "modelo"]
df = df.drop(columns=columnas_a_eliminar)

In [72]:
from sklearn.preprocessing import LabelEncoder

def convertir_categoricas_a_numericas(df):
    df_1 = df.copy()  # Crear una copia del DataFrame para no modificar el original
    
    # Obtener las columnas categóricas
    columnas_categoricas = df_1.select_dtypes(include=['object']).columns.tolist()
    
    # Aplicar LabelEncoder a las columnas categóricas
    label_encoder = LabelEncoder()
    for columna in columnas_categoricas:
        df_1[columna] = label_encoder.fit_transform(df_1[columna])
    
    return df_1


In [73]:
df = convertir_categoricas_a_numericas(df)

In [74]:
df

,Combustible,año,kms,precio,marca
0,0,2019,96.700,21.900,28
1,0,2020,46.776,42.995,7
2,3,2022,16.600,19.550,30
3,0,2016,30.000,27.900,44
4,3,2021,54.800,16.400,55
...,...,...,...,...,...
9995,3,2022,11.916,31.850,28
9996,0,2022,6.554,27.800,5
9997,3,2021,24.095,23.900,21
9998,3,2023,8.000,25.500,65


In [76]:
nombre_archivo = 'coches_4.csv'

# Crear el archivo CSV a partir del DataFrame
df.to_csv(nombre_archivo, index=False)

In [77]:
test_data = pd.read_csv(r"coches_4.csv")

test_df = test_data.copy()

# Definimos la variabel Target.
TARGET = "precio"

In [78]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Combustible  10000 non-null  int64  
 1   año          10000 non-null  int64  
 2   kms          10000 non-null  float64
 3   precio       9928 non-null   float64
 4   marca        10000 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 390.8 KB
